**Задача 1: сравнение предложений**

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

* кошки (животные)
* UNIX-утилита cat для вывода содержимого файлов
* версии операционной системы OS X, названные в честь семейства кошачьих

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

In [1]:
import os
import re
import numpy as np
from itertools import chain
import scipy.spatial

Выполните следующие шаги:

**[1].** Скачайте файл с предложениями (sentences.txt).

**[2].** Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().

In [2]:
with open('sentences.txt', 'r') as file:
    sentences = [line.strip().lower() for line in file]

**[3].** Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.

In [3]:
regexp = re.compile('[^a-z]')
sentence = [[word for word in re.split(regexp, line) if word] for line in sentences]

**[4].** Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.

In [4]:
words = {word: index for index, word in enumerate(set(chain(*sentence)))}

**[5].** Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.

In [5]:
matrix = np.zeros((len(sentences), len(words)))
matrix.shape

(22, 254)

In [6]:
for i in range(len(sentences)):
    for j in sentence[i]:
        matrix[i][words[j]] += 1

**[6].** Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.

In [7]:
distances = [(index, scipy.spatial.distance.cosine(matrix[0], matrix[index])) for index in range(len(sentences))]
sorted_distance = sorted(distances, key=lambda x: x[1])
print (sorted_distance[1][0], sorted_distance[2][0])

6 4


**[7].** Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).

In [8]:
with open('submission-1.txt', 'w') as file:
    file.write(' '.join([str(i[0]) for i in sorted_distance[1:3]]))

**[8].** Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

In [9]:
print (sentences[sorted_distance[0][0]])
print ('\n')
print (sentences[sorted_distance[1][0]])
print (sentences[sorted_distance[2][0]])

in comparison to dogs, cats have not undergone major changes during the domestication process.


domestic cats are similar in size to the other members of the genus felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).
in one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.


Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.